In [1]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, timedelta
import os
import time
import csv

In [2]:
# 본인의 Bitquery api키를 입력
api_key = 'BQYrRP58Ms7NzA5zTfO6DjtJDrrqOR3C'
headers = {
    'Content-Type': 'application/json',
    'X-API-KEY': api_key
}

# Ticker
Ticker = "SOL"

## 밑에 4가지 값을 조절가능!
# 시작 날짜와 종료 날짜 설정 (변수 정의) 일자 단위로만 추출 가능
start_date = "2020-06-25"
end_date = "2020-06-26"

# 페이지네이션을 위해 초기 offset 설정 # 이 부분은 안 건들이는 게 좋음!
offset = 0
batch_size = 25000

## JSON을 개별로 저장한 후에 CSV로 통합하는 코드 - 안정성을 위해 이 코드 이용

In [3]:
# Bitquery 엔드포인트
url = 'https://graphql.bitquery.io/'

# GraphQL 쿼리 템플릿 설정
query_template = '''
query ($limit: Int!, $offset: Int!, $from: ISO8601DateTime, $till: ISO8601DateTime) {
  solana(network: solana) {
    transfers(
      date: {since: $from, till: $till}
      options: {asc: "block.height", limit: $limit, offset: $offset}
      currency: {is: "SOL"}
    ) {
      receiver {
        address
      }
      sender {
        address
      }
      transaction {
        fee
      }
      block {
        hash
        height
        timestamp {
          time(format: "%Y-%m-%d %H:%M:%S")
        }
      }
      amount: amount(currency: {is: "SOL"})
    }
  }
}
'''


# JSON 파일 저장 디렉토리 설정
output_dir = f'{Ticker}_{start_date}_{end_date}_json_pages'
os.makedirs(output_dir, exist_ok=True)

page_number = 1

params = {
    "limit": batch_size,
    "offset": offset,
    "from": start_date,
    "till": end_date
}
while True:
    response = requests.post(url, headers=headers, json={'query': query_template, 'variables': params})

    if response.status_code == 200: # 문제 없을 경우 데이터가 추출
        data = response.json()
        if 'data' in data and 'solana' in data['data'] and 'transfers' in data['data']['solana']:
            transfers = data['data']['solana']['transfers']
            if not transfers:
                break  # 더 이상 데이터가 없으면 종료

            # JSON 파일로 저장
            with open(os.path.join(output_dir, f'{Ticker}_{start_date}_{end_date}_page_{page_number}.json'), 'w') as json_file:
                json.dump(transfers, json_file, indent=4)
            
            params["offset"] += params["limit"]  # 다음 페이지로 이동
            print(f"page {page_number} is complete!")
            page_number += 1
        else:
            break  # 데이터가 없으면 종료
    else:
        print(f'Query failed with status code {response.status_code}')
        print(response.text)
        break

print(f'Data successfully saved to JSON files in {output_dir}')

Data successfully saved to JSON files in SOL_2020-06-25_2020-06-26_json_pages


In [5]:
## JSON 파일을 CSV로 변환하는 코드 ##

# JSON 파일 저장 디렉토리 설정
input_dir = f'{Ticker}_{start_date}_{end_date}_json_pages'
output_csv = f'{Ticker}_{start_date}_{end_date}_TRX.csv'

# CSV 파일에 저장할 필드 이름 설정
fieldnames = [
    'block_timestamp', 'block_height', 'block_hash', 'sender_address', 
    'receiver_address', 'amount'
]

# CSV 파일 생성 및 필드 이름 작성
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # JSON 파일들 읽기
    for filename in os.listdir(input_dir):
        if filename.endswith('.json'):
            with open(os.path.join(input_dir, filename), 'r') as jsonfile:
                data = json.load(jsonfile)['data']['solana']['transfers']

                # JSON 데이터에서 필요한 필드 추출하여 CSV 파일에 쓰기
                for transfer in data:
                    row = {
                        'block_timestamp': transfer['block']['timestamp']['time'],
                        'block_height': transfer['block']['height'],
                        'block_hash': transfer['block']['hash'],
                        'sender_address': transfer['sender']['address'],
                        'receiver_address': transfer['receiver']['address'],
                        'currency_symbol': transfer['currency']['symbol'],
                        'amount': transfer['amount']
                    }
                    writer.writerow(row)

print(f'Data successfully saved to CSV file {output_csv}')

Data successfully saved to CSV file SOL_2023-12-01_2023-12-02_TRX.csv


## JSON가져와서 필요한 부분만 처리후 바로 CSV로 저장하는 코드(비추) - 별로 안커서 한번에 잘 될때 이 코드를 사용

In [5]:
# # Bitquery API key and endpoint

# url = 'https://graphql.bitquery.io/'

# # GraphQL query template
# query_template = '''
# query ($limit: Int!, $offset: Int!, $from: ISO8601DateTime, $till: ISO8601DateTime) {
#   solana(network: solana) {
#     transfers(
#       date: {since: $from, till: $till}
#       options: {asc: "block.height", limit: $limit, offset: $offset}
#     ) {
#       receiver {
#         address
#         type
#       }
#       sender {
#         address
#         type
#       }
#       block {
#         hash
#         height
#         timestamp {
#           time(format: "%Y-%m-%d %H:%M:%S")
#         }
#       }
#       currency {
#         symbol
#       }
#       amount: amount(currency: {is: "SOL"})
#     }
#   }
# }
# '''

# # Parameters
# limit = 10000
# offset = 0
# start_date = "2023-12-01"
# end_date = "2023-12-02T23:59:59"

# # Create directory for JSON pages
# input_dir = f'Solana_{start_date}_{end_date}_json_pages'
# os.makedirs(input_dir, exist_ok=True)

# # Headers
# headers = {
#     'Content-Type': 'application/json',
#     'X-API-KEY': api_key
# }

# # CSV output file
# output_csv = f'Solana_{start_date}-{end_date}_trx.csv'

# # CSV fieldnames
# fieldnames = [
#     'block_timestamp', 'block_height', 'sender_address', 'receiver_address',
#     'currency_symbol', 'amount', 'amount_usd', 'transaction_hash', 'external'
# ]

# # CSV file creation and fieldnames writing
# with open(output_csv, 'w', newline='') as csvfile:
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()

#     # Loop through pages
#     page_number = 1
#     while True:
#         # Format query with parameters
#         query = query_template % (network, limit, offset, start_date, end_date)

#         # Make API request
#         response = requests.post(url, headers=headers, json={'query': query})

#         if response.status_code == 200:
#             data = response.json()['data']['solana']['transactions']

#             # Save JSON to file
#             with open(os.path.join(input_dir, f'page_{page_number}.json'), 'w') as json_file:
#                 json.dump(data, json_file, indent=4)

#             # Write data to CSV
#             for transaction in data:
#                 row = {
#                     'block_timestamp': transaction['block']['timestamp']['time'],
#                     'block_height': transaction['block']['height'],
#                     'sender_address': transaction['sender']['address'],
#                     'receiver_address': transaction['receiver']['address'],
#                     'currency_symbol': transaction['currency']['symbol'],
#                     'amount': transaction['amount'],
#                     'amount_usd': transaction['amount_usd'],
#                     'transaction_hash': transaction['transaction']['hash'],
#                     'external': transaction['external']
#                 }
#                 writer.writerow(row)

#             # Move to next page
#             offset += limit
#             page_number += 1

#             # If no more data, break the loop
#             if len(data) < limit:
#                 break

#         else:
#             print(f'Query failed with status code {response.status_code}')
#             print(response.text)
#             break

# print(f'Data successfully saved to CSV file {output_csv}')


ValueError: unsupported format character 'Y' (0x59) at index 360

만개를 처리하는 데 약 20초가 걸리고, 하루 기준 용량이 약 60MB. 이는 일주일이면 400MB가 넘는다. 아래 코드가 약간의 시간이 더 걸리는 것으로 보인다.